# 1. Import Libraries 

In [1]:
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from rdflib.util import guess_format

from SPARQLWrapper import SPARQLWrapper, JSON, N3

import pandas as pd
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

#Used from INM713 Semantic Web Technologies and Knowledge Graphs Lab 5
from isub import isub
from lookup import DBpediaLookup

import csv
import owlrl

import math

C:\Users\Abhimanyu Acharya\anaconda3\envs\diss_kg\lib\site-packages\rdflib_jsonld\__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,


# 2. Import Data

### 2.1 Import class attributes and save as list

In [2]:
predicates_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicates.txt", sep = '\t',header = None, names = ['predicates'])
predicate_list = predicates_df['predicates'].tolist()
print(predicate_list)

['black', 'white', 'blue', 'brown', 'gray', 'orange', 'red', 'yellow', 'patches', 'spots', 'stripes', 'furry', 'hairless', 'toughskin', 'big', 'small', 'bulbous', 'lean', 'flippers', 'hands', 'hooves', 'pads', 'paws', 'longleg', 'longneck', 'tail', 'chewteeth', 'meatteeth', 'buckteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'smelly', 'flys', 'hops', 'swims', 'tunnels', 'walks', 'fast', 'slow', 'strong', 'weak', 'muscle', 'bipedal', 'quadrapedal', 'active', 'inactive', 'nocturnal', 'hibernate', 'agility', 'fish', 'meat', 'plankton', 'vegetation', 'insects', 'forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker', 'newworld', 'oldworld', 'arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean', 'ground', 'water', 'tree', 'cave', 'fierce', 'timid', 'smart', 'group', 'solitary', 'nestspot', 'domestic']


### 2.2. Import Predicate Matrix

### 2.2.1 Binary

In [3]:
predicateMatrixBinary_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/predicate-matrix-binary.txt", sep = " ",header = None, names = predicate_list)
predicateMatrixBinary_df.head()

,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic
0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0
2,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0
4,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1


### 2.3 Load csv with animal classes

In [4]:
classes_df = pd.read_csv("C:/Dissertation/Data/AwA2-data/Animals_with_Attributes2/classes.txt", sep = '\t',header = None, names = ['labels','classes'])
classes_df.head()

,labels,classes
0,1,antelope
1,2,grizzly+bear
2,3,killer+whale
3,4,beaver
4,5,dalmatian


### 2.4 Rename animal classes

In [5]:
classes_list = ["Antelope","Grizzly bear","Killer Whale","Beaver", "Dalmatian (dog)", "Persian cat", "Horse", "German Shepherd", 
               "Blue whale", "Siamese cat", "Skunk", "Mole (animal)", "Tiger", "Hippopotamus", "Leopard", "Moose", 
               "Spider monkey", "Humpback whale", "Elephant", "Gorilla", "Ox", "Fox", "Sheep", "Seal", "Chimpanzee", 
                "Hamster", "Squirrel", "Rhinoceros","Rabbit", "Bat", "Giraffe", "Wolf", "Chihuahua (dog)",
               "Rat", "Weasel", "Otter", "Buffalo", "Zebra", "Giant panda","Deer", "Bobcat", "Pig", "Lion", "Mouse",
                "Polar bear", "Collie", "Walrus", "Raccoon", "Cattle", "Dolphin"]

### 2.5  Concat Class labels and predicate matrix

In [6]:
predicate_df = pd.concat([classes_df, predicateMatrixBinary_df], axis=1)
predicate_df['corrected_class_names'] = classes_list
predicate_df

,labels,classes,black,white,blue,brown,gray,orange,red,yellow,patches,spots,stripes,furry,hairless,toughskin,big,small,bulbous,lean,flippers,hands,hooves,pads,paws,longleg,longneck,tail,chewteeth,meatteeth,buckteeth,strainteeth,horns,claws,tusks,smelly,flys,hops,swims,tunnels,walks,fast,slow,strong,weak,muscle,bipedal,quadrapedal,active,inactive,nocturnal,hibernate,agility,fish,meat,plankton,vegetation,insects,forager,grazer,hunter,scavenger,skimmer,stalker,newworld,oldworld,arctic,coastal,desert,bush,plains,forest,fields,jungle,mountains,ocean,ground,water,tree,cave,fierce,timid,smart,group,solitary,nestspot,domestic,corrected_class_names
0,1,antelope,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,Antelope
1,2,grizzly+bear,1,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,1,0,0,Grizzly bear
2,3,killer+whale,1,1,0,0,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,Killer Whale
3,4,beaver,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0,Beaver
4,5,dalmatian,1,1,0,0,0,0,0,0,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,1,Dalmatian (dog)
5,6,persian+cat,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,1,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,Persian cat
6,7,horse,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,1,1,0,0,1,Horse
7,8,german+shepherd,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,German Shepherd
8,9,blue+whale,0,0,1,0,1,0,0,0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0,Blue whale
9,10,siamese+cat,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,1,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,Siamese cat


### 2.6 Store attributes/column names as a list

In [7]:
attributes = []
for col in predicate_df.columns:
    attributes.append(col)
attributes.remove('labels')
attributes.remove('classes')
attributes.remove('corrected_class_names')

In [8]:
print(attributes)

['black', 'white', 'blue', 'brown', 'gray', 'orange', 'red', 'yellow', 'patches', 'spots', 'stripes', 'furry', 'hairless', 'toughskin', 'big', 'small', 'bulbous', 'lean', 'flippers', 'hands', 'hooves', 'pads', 'paws', 'longleg', 'longneck', 'tail', 'chewteeth', 'meatteeth', 'buckteeth', 'strainteeth', 'horns', 'claws', 'tusks', 'smelly', 'flys', 'hops', 'swims', 'tunnels', 'walks', 'fast', 'slow', 'strong', 'weak', 'muscle', 'bipedal', 'quadrapedal', 'active', 'inactive', 'nocturnal', 'hibernate', 'agility', 'fish', 'meat', 'plankton', 'vegetation', 'insects', 'forager', 'grazer', 'hunter', 'scavenger', 'skimmer', 'stalker', 'newworld', 'oldworld', 'arctic', 'coastal', 'desert', 'bush', 'plains', 'forest', 'fields', 'jungle', 'mountains', 'ocean', 'ground', 'water', 'tree', 'cave', 'fierce', 'timid', 'smart', 'group', 'solitary', 'nestspot', 'domestic']


# 3. URI generation & RDF generation

### 3.1 Set up dictionary to hold the URI's
Adapted from N3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5 solution

In [9]:
#Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
stringToURI = dict()

## 3.2 Set up empty graph and Namespace

In [10]:
#Create empty graph
g = Graph()

#Define Namespace
awa2_onto_uri = "http://www.semanticweb.org/inm363/AwA2#"
      
AwA2 = Namespace(awa2_onto_uri)
dbo = Namespace("https://dbpedia.org/ontology/") 
owl = Namespace("http://www.w3.org/2002/07/owl#") 

#Bind Prefixes 
g.bind("AwA2", AwA2) #cw is a newly created namespace  
g.bind("dbo", dbo) 
g.bind("owl", owl) 

## 3.3 Define Code to generate Triples
Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.3.1 Modular Mapping functions 

In [11]:
# Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

def is_nan( x):
        return (x != x)         
            
        
def createURIForEntity(name, useExternalURI):
    stringToURI[name] = AwA2 + name.replace(" ", "_")
    
    if useExternalURI is True: #We connect to online KG
        uri = getExternalKGURI(name)
        if uri!="":
            stringToURI[name]=uri
    
    return stringToURI[name]

  
def getExternalKGURI(name):
    entities = DBpediaLookup().getKGEntities(name, 2)
    current_sim = -1
    current_uri=''
    for ent in entities:           
        isub_score = isub(name, ent.label) 
        if current_sim < isub_score:
            current_uri = ent.ident
            current_sim = isub_score    
        
    return current_uri 

    
def mappingToCreateTypeTriple(dataframe, subject_column, class_type, useExternalURI):
        
        for subject in dataframe[subject_column]:
                
            if subject.lower() in stringToURI:
                entity_uri = stringToURI[subject.lower()]
            else:
                entity_uri = createURIForEntity(subject.lower(), useExternalURI)
            
            g.add((URIRef(entity_uri), RDF.type, class_type))
            print('Example genrated URI for',subject_column,' ',URIRef(entity_uri))
            print('Example generated rdf:type triple for',subject_column,' ',URIRef(entity_uri),RDF.type, class_type)
            print('')       


def mappingToCreateLiteralTriple(dataframe, subject_column, object_column, predicate, datatype):
        
        for subject, lit_value in zip(dataframe[subject_column], dataframe[object_column]):
            
            if is_nan(lit_value) or lit_value==None or lit_value=="":
                pass
            
            else:
                #Uri as already created
                entity_uri = stringToURI[subject.lower()]
                    
                #Literal
                lit = Literal(lit_value, datatype=datatype)
                
                #New triple
                g.add((URIRef(entity_uri), predicate, lit))                     
                                   
            print('Example generated data property triple for',subject_column,'and',object_column,':', URIRef(entity_uri), predicate, lit)
            print('')
                         
            
def mappingToCreateObjectTriple(dataframe, subject_column, object_name, predicate):
        
    for subject in dataframe[subject_column]:
            
        if is_nan(object):
            pass
            
        else:
            #Uri as already created
            subject_uri = stringToURI[subject.lower()]
            object_uri = object_name
                    
            #New triple
            g.add((URIRef(subject_uri), predicate, URIRef(object_uri)))
        print('Example generated object property triple for',URIRef(subject_uri), predicate, URIRef(object_uri))
        print('')

# 4 Define Class Type Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 4.1 Animal classes

In [12]:
stringToURI.clear() #empty dict to remove all URIs from DBPedia
mappingToCreateTypeTriple(predicate_df,'classes', AwA2.Animal, False) #Comment to use user define URI

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#antelope
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#grizzly+bear
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#killer+whale
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#Animal

Example genrated URI for classes   http://www.semanticweb.org/inm363/AwA2#beaver
Example generated rdf:type triple for classes   http://www.semanticweb.org/inm363

### 4.2 Animal attributes

In [13]:
attribute_classes = [AwA2.AnimalbyColour, AwA2.AnimalbySkinPattern, AwA2.AnimalbyFurType, AwA2.AnimalbyBodyType, AwA2.AnimalbyAppendageType,
AwA2.AnimalbyTeethType, AwA2.AnimalbyMovementType, AwA2.AnimalbyMovementSpeed, AwA2.AnimalbyStrengthType,AwA2.AnimalbyLocomotionType,
AwA2.AnimalbyActivityLevel, AwA2.AnimalbyActivityPeriod, AwA2.AnimalbyDietType,AwA2.AnimalbyMethodOfEating, AwA2.AnimalbyWorldType,
AwA2.AnimalbyHabitatType, AwA2.AnimalbyDwellingType, AwA2.AnimalbyTemparmentType, AwA2.AnimalbyGroupBehavior, AwA2.AnimalbyUniqueFeature,
AwA2.AnimalbyOdour, AwA2.AnimalbyIntelligenceType,  AwA2.AnimalbyLifeType, AwA2.AnimalbyChildcare, AwA2.AnimalbyMovementSkill, AwA2.AnimalbyBehavioralAdaptation]

for i in attribute_classes:
    g.add((i, RDFS.subClassOf, AwA2.Animal))
print('Example generated rdf:type triple for',i, RDFS.subClassOf, AwA2.Animal)


Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#AnimalbyBehavioralAdaptation http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AwA2#Animal


### 4.2.1 Colour

In [14]:
colour = {'black':AwA2.BlackAnimal, 'white':AwA2.WhiteAnimal, 'blue':AwA2.BlueAnimal, 'brown':AwA2.BrownAnimal, 
               'gray':AwA2.GrayAnimal, 'orange':AwA2.OrangeAnimal, 'red':AwA2.RedAnimal, 'yellow':AwA2.YellowAnimal}

for i in colour:
    g.add((colour[i], RDFS.subClassOf, AwA2.AnimalbyColour))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, colour[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, colour[i])       

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BlackAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#WhiteAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BlueAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BrownAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#GrayAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#bobcat http:/

### 4.2.2 Skin Pattern

In [15]:
skin_type = {'patches':AwA2.PatchySkinAnimal, 'spots':AwA2.SpotSkinAnimal, 'stripes':AwA2.StripeSkinAnimal, 
                  'toughskin':AwA2.ToughSkinAnimal}

for i in skin_type:
    g.add((skin_type[i], RDFS.subClassOf, AwA2.AnimalbySkinPattern))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, skin_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, skin_type[i])                 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#PatchySkinAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SpotSkinAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#StripeSkinAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ToughSkinAnimal


### 4.2.3 Fur

In [16]:
fur_type = {'furry':AwA2.FurryAnimal, 'hairless':AwA2.HairlessAnimal}

for i in fur_type:
    g.add((fur_type[i], RDFS.subClassOf, AwA2.AnimalbyFurType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, fur_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, fur_type[i])    

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FurryAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HairlessAnimal


### 4.2.4 Body

In [17]:
body_type = {'big':AwA2.BigBodyAnimal, 'small':AwA2.SmallBodyAnimal, 'bulbous':AwA2.BulbousBodyAnimal, 'lean':AwA2.LeanBodyAnimal, 
             'muscle':AwA2.MuscularBodyAnimal}

for i in body_type:
    g.add((body_type[i], RDFS.subClassOf, AwA2.AnimalbyBodyType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, body_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, body_type[i])     

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BigBodyAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SmallBodyAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BulbousBodyAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#LeanBodyAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#MuscularBodyAnimal


### 4.2.5 Appendage

In [18]:
appendage_type = {'flippers':AwA2.AnimalwithFlippers, 'hands':AwA2.AnimalwithHands, 'hooves':AwA2.AnimalwithHooves,
                  'pads':AwA2.AnimalwithPads, 'paws':AwA2.AnimalwithPaws}

for i in appendage_type:
    g.add((appendage_type[i], RDFS.subClassOf, AwA2.AnimalbyAppendageType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, appendage_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, appendage_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalwithFlippers
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#chimpanzee http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalwithHands
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalwithHooves
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalwithPads
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalwithPaws


### 4.2.6 Teeth

In [19]:
teeth_type = {'chewteeth':AwA2.AnimalswithChewTeeth, 'meatteeth':AwA2.AnimalswithMeanTeeth, 'buckteeth':AwA2.AnimalswithBuckTeeth, 
              'strainteeth':AwA2.AnimalswithStrainTeeth}

for i in teeth_type:
    g.add((teeth_type[i], RDFS.subClassOf, AwA2.AnimalbyTeethType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, teeth_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, teeth_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalswithChewTeeth
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalswithMeanTeeth
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#walrus http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalswithBuckTeeth
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#walrus http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalswithStrainTeeth


### 4.2.7 Movement

In [20]:
movement_type = {'flys':AwA2.FlyingAnimal, 'hops':AwA2.HoppingAnimal, 'swims':AwA2.SwimmingAnimal, 'tunnels':AwA2.TunnelingAnimal, 
                 'walks':AwA2.WalkingAnimal}

for i in movement_type:
    g.add((movement_type[i], RDFS.subClassOf, AwA2.AnimalbyMovementType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, movement_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, movement_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#bat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FlyingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#rabbit http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HoppingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SwimmingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#mouse http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#TunnelingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#WalkingAnimal


### 4.2.8 Movement Speed

In [21]:
speed_type = {'fast':AwA2.FastMovingAnimal,'slow':AwA2.SlowMovingAnimal}

for i in speed_type:
    g.add((speed_type[i], RDFS.subClassOf, AwA2.AnimalbyMovementSpeed))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, speed_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, speed_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FastMovingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SlowMovingAnimal


### 4.2.9 Strength

In [22]:
strength_type = {'strong':AwA2.StrongAnimal,'weak':AwA2.WeakAnimal}

for i in strength_type:
    g.add((strength_type[i], RDFS.subClassOf, AwA2.AnimalbyStrengthType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, strength_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, strength_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#StrongAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#mouse http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#WeakAnimal


### 4.2.10 Locomotion

In [23]:
locomotion_type = {'bipedal':AwA2.BipedalAnimal, 'quadrapedal':AwA2.QuadrapedalAnimal}

for i in locomotion_type:
    g.add((locomotion_type[i], RDFS.subClassOf, AwA2.AnimalbyLocomotionType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, locomotion_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, locomotion_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#polar+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BipedalAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#QuadrapedalAnimal


### 4.2.11 Activity Level

In [24]:
activity_level = {'active':AwA2.ActiveAnimal, 'inactive':AwA2.InactiveAnimal}

for i in activity_level:
    g.add((activity_level[i], RDFS.subClassOf, AwA2.AnimalbyActivityLevel))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, activity_level[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, activity_level[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ActiveAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#InactiveAnimal


### 4.2.12 Activity Period

In [25]:
activity_period = {'nocturnal':AwA2.NocturnalAnimal}

for i in activity_period:
    g.add((activity_period[i], RDFS.subClassOf, AwA2.AnimalbyActivityPeriod))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, activity_period[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, activity_period[i])   

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#NocturnalAnimal


### 4.2.13 Diet 

In [26]:
diet_type = {'fish':AwA2.FishEatingAnimal, 'meat':AwA2.MeatEatingAnimal, 'plankton':AwA2.PlanktonEatingAnimal, 'vegetation':AwA2.VegetationEatingAnimal, 
             'insects':AwA2.InsectEatingAnimal}

for i in diet_type:
    g.add((diet_type[i], RDFS.subClassOf, AwA2.AnimalbyDietType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, diet_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, diet_type[i])   

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FishEatingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#MeatEatingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#humpback+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#PlanktonEatingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#VegetationEatingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#rat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#InsectEatingAnimal


### 4.2.14 Method of Eating

In [27]:
method_of_eating = {'forager':AwA2.ForagingAnimals, 'grazer':AwA2.GrazingAnimals, 'hunter':AwA2.HuntingAnimals,
                        'scavenger':AwA2.ScavengingAnimals, 'skimmer':AwA2.SkimmingAnimals, 'stalker':AwA2.StalkingAnimals}

for i in method_of_eating:
    g.add((method_of_eating[i], RDFS.subClassOf, AwA2.AnimalbyMethodOfEating))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, method_of_eating[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, method_of_eating[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ForagingAnimals
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#GrazingAnimals
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#collie http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HuntingAnimals
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ScavengingAnimals
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#humpback+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SkimmingAnimals
Example generated rdf:type triple for http://www.semanticw

### 4.2.15  World

In [28]:
world_type = {'newworld':AwA2.NewWorldAnimal,'oldworld':AwA2.OldWorldAnimal}

for i in world_type:
    g.add((world_type[i], RDFS.subClassOf, AwA2.AnimalbyWorldType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, world_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, world_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#NewWorldAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#OldWorldAnimal


### 4.2.16 Habitat

In [29]:
habitat_type = {'arctic':AwA2.ArticAnimal, 'coastal':AwA2.CoastalAnimal, 'desert':AwA2.DesertAnimal, 
                'bush':AwA2.BushAnimal, 'plains':AwA2.PlainAnimal, 'forest':AwA2.ForestAnimal, 
                'fields':AwA2.FieldAnimal, 'jungle':AwA2.JungleAnimal, 'mountains':AwA2.MountainAnimal, 
                'ocean':AwA2.OceanAnimal}

for i in habitat_type:
    g.add((habitat_type[i], RDFS.subClassOf, AwA2.AnimalbyHabitatType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, habitat_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, habitat_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#walrus http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ArticAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#CoastalAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#lion http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#DesertAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#lion http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BushAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#PlainAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http

### 4.2.17 Dwelling Type

In [30]:
home_type = {'ground':AwA2.GroundDwellingAnimal, 'water':AwA2.WaterDwellingAnimal, 'tree':AwA2.TreeDwellingAnimal, 
             'cave':AwA2.CaveDwellingAnimal}

for i in home_type:
    g.add((home_type[i], RDFS.subClassOf, AwA2.AnimalbyDwellingType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, home_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, home_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#GroundDwellingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#WaterDwellingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#TreeDwellingAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#bobcat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#CaveDwellingAnimal


### 4.2.18 Temparment

In [31]:
temperment_type = {'fierce':AwA2.FierceAnimal, 'timid':AwA2.TimidAnimal}

for i in temperment_type:
    g.add((temperment_type[i], RDFS.subClassOf, AwA2.AnimalbyTemparmentType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, temperment_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, temperment_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FierceAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#TimidAnimal


### 4.2.19 Group behavior

In [32]:
group_type = {'group':AwA2.GroupAnimal,'solitary':AwA2.SolitaryAnimal}

for i in group_type:
    g.add((group_type[i], RDFS.subClassOf, AwA2.AnimalbyGroupBehavior))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, group_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, group_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#GroupAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SolitaryAnimal


### 4.2.20 Unique Physical Feature

In [33]:
unique_feature = {'longleg':AwA2.LongLegAnimal, 'longneck':AwA2.LongNeckAnimal, 'horns':AwA2.HornedAnimal, 
                  'claws':AwA2.ClawedAnimal, 'tusks':AwA2.TuskedAnimal}

for i in unique_feature:
    g.add((unique_feature[i], RDFS.subClassOf, AwA2.AnimalbyUniqueFeature))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, unique_feature[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, unique_feature[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#deer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#LongLegAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#deer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#LongNeckAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HornedAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#ClawedAnimal
Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#walrus http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#TuskedAnimal


### 4.2.21 Body Odour

In [34]:
odour_type = {'smelly':AwA2.SmellyAnimal}

for i in odour_type:
    g.add((odour_type[i], RDFS.subClassOf, AwA2.AnimalbyOdour))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, odour_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, odour_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#cow http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SmellyAnimal


### 4.2.22 Inteligence

In [35]:
intelligence_type = {'smart':AwA2.SmartAnimal}

for i in intelligence_type:
    g.add((intelligence_type[i], RDFS.subClassOf, AwA2.AnimalbyIntelligenceType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, intelligence_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, intelligence_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SmartAnimal


### 4.2.23 LifeType

In [36]:
animal_type = {'domestic':AwA2.DomesticAnimal}

for i in animal_type:
    g.add((animal_type[i], RDFS.subClassOf, AwA2.AnimalbyLifeType))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, animal_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, animal_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#DomesticAnimal


### 4.2.24 Childcare

In [37]:
childcare_type = {'nestspot':AwA2.NestSpot}

for i in childcare_type:
    g.add((childcare_type[i], RDFS.subClassOf, AwA2.AnimalbyChildcare))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, childcare_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, childcare_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#NestSpot


### 4.2.25 Movement Skill

In [38]:
phy_coord_type = {'agility':AwA2.AgileAnimal}

for i in phy_coord_type:
    g.add((phy_coord_type[i], RDFS.subClassOf, AwA2.AnimalbyMovementSkill))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, phy_coord_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, phy_coord_type[i]) 

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#dolphin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AgileAnimal


### 4.2.26 Behavioral Adaptation

In [39]:
adaptation_type = {'hibernate':AwA2.HibernatingAnimal}

for i in adaptation_type:
    g.add((adaptation_type[i], RDFS.subClassOf, AwA2.AnimalbyBehavioralAdaptation))
    for animal in predicate_df.loc[predicate_df[i]==1]['classes'].values:
        g.add((URIRef(stringToURI[animal]), RDF.type, adaptation_type[i]))
       
    print('Example generated rdf:type triple for',URIRef(stringToURI[animal]), RDF.type, adaptation_type[i])  

Example generated rdf:type triple for http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HibernatingAnimal


### 3.5 Create Litral type Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

In [40]:
mappingToCreateLiteralTriple(predicate_df, 'classes', 'corrected_class_names',AwA2.animalNames, XSD.string)

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#antelope http://www.semanticweb.org/inm363/AwA2#animalNames Antelope

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#grizzly+bear http://www.semanticweb.org/inm363/AwA2#animalNames Grizzly bear

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#killer+whale http://www.semanticweb.org/inm363/AwA2#animalNames Killer Whale

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#beaver http://www.semanticweb.org/inm363/AwA2#animalNames Beaver

Example generated data property triple for classes and corrected_class_names : http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.semanticweb.org/inm363/AwA2#animalNames Dalmatian (dog)

Example generated data property triple f

## 4. Print all triples in the  generated RDF data

In [41]:
print("The graph contains '" + str(len(g)) + "' triples.")
for subj, pred, obj in g:
    print(subj, pred, obj)

The graph contains '1733' triples.
http://www.semanticweb.org/inm363/AwA2#horse http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#StrongAnimal
http://www.semanticweb.org/inm363/AwA2#giraffe http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AnimalswithChewTeeth
http://www.semanticweb.org/inm363/AwA2#blue+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#SwimmingAnimal
http://www.semanticweb.org/inm363/AwA2#humpback+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#StrongAnimal
http://www.semanticweb.org/inm363/AwA2#leopard http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#StalkingAnimals
http://www.semanticweb.org/inm363/AwA2#deer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AgileAnimal
http://www.semanticweb.org/inm363/AwA2#sheep http://www.w3.org/199

http://www.semanticweb.org/inm363/AwA2#zebra http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#AgileAnimal
http://www.semanticweb.org/inm363/AwA2#bat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FierceAnimal
http://www.semanticweb.org/inm363/AwA2#dalmatian http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#WalkingAnimal
http://www.semanticweb.org/inm363/AwA2#ox http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#BulbousBodyAnimal
http://www.semanticweb.org/inm363/AwA2#raccoon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#FishEatingAnimal
http://www.semanticweb.org/inm363/AwA2#lion http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AwA2#HuntingAnimals
http://www.semanticweb.org/inm363/AwA2#bat http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/i

## 5. Perform Reasoning

In [42]:
print("Data triples before reasoning: '" + str(len(g)) + "'.")

g_expanded = g

##Perform reasoning
#owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=False, datatype_axioms=False).expand(g_expanded)
#print("Data triples after reasoning: '" + str(len(g_expanded)) + "'.")

Data triples before reasoning: '1733'.


In [43]:
#print("The graph contains '" + str(len(g_expanded)) + "' triples.")
#for subj, pred, obj in g_expanded:
    #print(subj, pred, obj)

# 6. Save the Generated RDF data 

In [44]:
g.serialize(destination='AWA2_KG01_basic_attributes_03.ttl', format='ttl') 
g.serialize(destination='C:/Users/Abhimanyu Acharya/Documents/GitHub/Dissertation/OWL2Vec-Star-master/dissertation/AWA2_KG01_basic_attributes_03.owl', format='xml') 


## 7. Run some queries to check the generated RDF data

In [45]:
g_test = Graph()

#Load Generated RDF
Generated_rdf = "AWA2_KG01_basic_attributes_03.ttl" # define path to Generated RDF
g_test.load(Generated_rdf ,  format=guess_format(Generated_rdf)) #load Generated RDF

In [46]:
qres = g_test.query("""
                SELECT DISTINCT ?animals WHERE {
                ?animals rdf:type AwA2:Animal .  
                ?animals rdf:type AwA2:WhiteAnimal . 
                ?animals rdf:type AwA2:GroupAnimal .}    
            """)


print("%s animals satisfying the query." % (str(len(qres))))

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    line_str = row.animals
    print(line_str)

14 animals satisfying the query.
http://www.semanticweb.org/inm363/AwA2#wolf
http://www.semanticweb.org/inm363/AwA2#horse
http://www.semanticweb.org/inm363/AwA2#pig
http://www.semanticweb.org/inm363/AwA2#sheep
http://www.semanticweb.org/inm363/AwA2#seal
http://www.semanticweb.org/inm363/AwA2#zebra
http://www.semanticweb.org/inm363/AwA2#tiger
http://www.semanticweb.org/inm363/AwA2#dolphin
http://www.semanticweb.org/inm363/AwA2#cow
http://www.semanticweb.org/inm363/AwA2#rabbit
http://www.semanticweb.org/inm363/AwA2#giant+panda
http://www.semanticweb.org/inm363/AwA2#dalmatian
http://www.semanticweb.org/inm363/AwA2#killer+whale
http://www.semanticweb.org/inm363/AwA2#mouse


In [47]:
qres = g.query("""
                SELECT ?attribute WHERE {
                ?attribute rdfs:subClassOf AwA2:Animal .
                }
               """)


print("%s classes satisfying the query." % (str(len(qres))))

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    line_str = row.attribute
    print(line_str)

26 classes satisfying the query.
http://www.semanticweb.org/inm363/AwA2#AnimalbyMethodOfEating
http://www.semanticweb.org/inm363/AwA2#AnimalbyActivityPeriod
http://www.semanticweb.org/inm363/AwA2#AnimalbyFurType
http://www.semanticweb.org/inm363/AwA2#AnimalbyHabitatType
http://www.semanticweb.org/inm363/AwA2#AnimalbyMovementSkill
http://www.semanticweb.org/inm363/AwA2#AnimalbyIntelligenceType
http://www.semanticweb.org/inm363/AwA2#AnimalbyColour
http://www.semanticweb.org/inm363/AwA2#AnimalbyDwellingType
http://www.semanticweb.org/inm363/AwA2#AnimalbyTeethType
http://www.semanticweb.org/inm363/AwA2#AnimalbyLifeType
http://www.semanticweb.org/inm363/AwA2#AnimalbyBehavioralAdaptation
http://www.semanticweb.org/inm363/AwA2#AnimalbyUniqueFeature
http://www.semanticweb.org/inm363/AwA2#AnimalbyChildcare
http://www.semanticweb.org/inm363/AwA2#AnimalbyWorldType
http://www.semanticweb.org/inm363/AwA2#AnimalbyActivityLevel
http://www.semanticweb.org/inm363/AwA2#AnimalbyDietType
http://www.semant